In [6]:
from skimage.feature import hog, local_binary_pattern
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

# Feature extractions

In [9]:
data = pd.read_csv('data/train.csv')
data.head()

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
0,0,Men Tshirts,5,default,round,printed,default,short sleeves,NaN,NaN,NaN,NaN,NaN
1,1,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
2,2,Men Tshirts,5,default,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
3,3,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
4,4,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN


In [2]:
img_path = 'data/train_images'

In [18]:
# resize images to 360x360 - padding with zeros
if not os.path.exists('resized_images'):           # create a new directory to store resized images
    os.makedirs('resized_images')

for file in os.listdir(img_path):
    img = cv2.imread(os.path.join(img_path, file), cv2.IMREAD_GRAYSCALE)
    h, w = img.shape[:2]
    scale = 360 / float(max(h, w))
    new_h, new_w = int(h * scale), int(w * scale)
    resized_image = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

    pad_h = (360 - new_h) // 2
    pad_w = (360 - new_w) // 2

    padded_image = cv2.copyMakeBorder(resized_image, pad_h, 360 - new_h - pad_h, pad_w, 360 - new_w - pad_w,
                                        cv2.BORDER_CONSTANT, value=0)
    
    cv2.imwrite('resized_images/' + file, padded_image)
    print('Resized', file)


Resized 000000.jpg
Resized 000001.jpg
Resized 000002.jpg
Resized 000003.jpg
Resized 000004.jpg
Resized 000005.jpg
Resized 000006.jpg
Resized 000007.jpg
Resized 000008.jpg
Resized 000009.jpg
Resized 000010.jpg
Resized 000011.jpg
Resized 000012.jpg
Resized 000013.jpg
Resized 000014.jpg
Resized 000015.jpg
Resized 000016.jpg
Resized 000017.jpg
Resized 000018.jpg
Resized 000019.jpg
Resized 000020.jpg
Resized 000021.jpg
Resized 000022.jpg
Resized 000023.jpg
Resized 000024.jpg
Resized 000025.jpg
Resized 000026.jpg
Resized 000027.jpg
Resized 000028.jpg
Resized 000029.jpg
Resized 000030.jpg
Resized 000031.jpg
Resized 000032.jpg
Resized 000033.jpg
Resized 000034.jpg
Resized 000035.jpg
Resized 000036.jpg
Resized 000037.jpg
Resized 000038.jpg
Resized 000039.jpg
Resized 000040.jpg
Resized 000041.jpg
Resized 000042.jpg
Resized 000043.jpg
Resized 000044.jpg
Resized 000045.jpg
Resized 000046.jpg
Resized 000047.jpg
Resized 000048.jpg
Resized 000049.jpg
Resized 000050.jpg
Resized 000051.jpg
Resized 0000

KeyboardInterrupt: 

In [ ]:
def extract_hog_features(img, bins=8):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hog_features, _ = hog(img, orientations=9, pixels_per_cell=(8, 8),
                            cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys', visualize=True)
    hog_features = hog_features.flatten()
    hist, _ = np.histogram(hog_features, bins=bins)
    hist = hist.astype('float')
    # hist /= (hist.sum() + 1e-7)
    return hist

def extract_lbp_features(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(img, 8, 1, method='uniform')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, lbp.max() + 1), range=(0, lbp.max()))
    hist = hist.astype('float')
    # hist /= (hist.sum() + 1e-7)
    return hist

def extract_canny_features(img, bins=8):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(img, 100, 200)
    hist, _ = np.histogram(edges.ravel(), bins=bins, range=(0, 256))
    hist = hist.astype('float')
    # hist /= (hist.sum() + 1e-7)
    return hist

def extract_bounding_box_contour(image, bins=20):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)
    
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        bounding_box = (x, y, w, h)
        silhouette = np.zeros_like(image)
        cv2.drawContours(silhouette, [largest_contour], -1, (255, 255, 255), thickness=cv2.FILLED)
    else:
        bounding_box = (0, 0, 0, 0)
        silhouette = np.zeros_like(image)

    silhouette = silhouette.flatten()
    hist_silhouette, _ = np.histogram(silhouette, bins=bins, range=(0, 256))
    hist_silhouette = hist_silhouette.astype('float')
    # hist_silhouette /= (hist_silhouette.sum() + 1e-7)

    return bounding_box, hist_silhouette

def extract_color_histogram(image, bins=20):
    channels = cv2.split(image)
    
    hist_features = []
    for channel in channels:
        hist = cv2.calcHist([channel], [0], None, [bins], [0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        hist_features.extend(hist)

    red = hist_features[:bins]
    green = hist_features[bins:2*bins]
    blue = hist_features[2*bins:]

    return red, green, blue

def extract_sift_features(image, bins=8):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray_image, None)
    
    if descriptors is not None:
        hist, _ = np.histogram(descriptors, bins=bins, range=(0, 256))
        hist = hist.astype('float')
        # hist /= (hist.sum() + 1e-7)
        return hist
    else:
        return np.zeros(bins)
    
def extract_hu_moments(img, bins=8):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    moments = cv2.moments(gray)
    hu_moments = cv2.HuMoments(moments).flatten()
    
    # Bin Hu Moments
    hist, _ = np.histogram(hu_moments, bins=bins)
    hist = hist.astype('float')
    # hist /= (hist.sum() + 1e-7)
    
    return hist


def extract_fourier_transform(img, bins=8):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Perform 2D Fourier Transform
    f_transform = np.fft.fft2(gray)
    f_shift = np.fft.fftshift(f_transform)
    
    # Take the magnitude spectrum
    magnitude_spectrum = np.log(np.abs(f_shift) + 1e-7)
    
    # Bin the magnitude spectrum
    hist, _ = np.histogram(magnitude_spectrum.ravel(), bins=bins)
    hist = hist.astype('float')
    # hist /= (hist.sum() + 1e-7)
    
    return hist


def extract_scharr_features(img, bins=8):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Scharr filter on X and Y directions
    scharr_x = cv2.Scharr(gray, cv2.CV_64F, 1, 0)
    scharr_y = cv2.Scharr(gray, cv2.CV_64F, 0, 1)
    
    # Magnitude of gradients
    scharr_magnitude = np.sqrt(scharr_x ** 2 + scharr_y ** 2)
    
    # Bin the gradient magnitudes
    hist, _ = np.histogram(scharr_magnitude.ravel(), bins=bins)
    hist = hist.astype('float')
    hist /= (hist.sum() + 1e-7)
    
    return hist


In [ ]:
new_data = []

for file in os.listdir('resized_images'):
    img = cv2.imread('resized_images/' + file)

    # hog_features = extract_hog_features(img)
    # lbp_features = extract_lbp_features(img)
    # canny_features = extract_canny_features(img)
    bounding_box, silhouette = extract_bounding_box_contour(img)
    red, green, blue = extract_color_histogram(img)
    # sift_features = extract_sift_features(img)
    # hu_moments = extract_hu_moments(img)
    # fourier_transform = extract_fourier_transform(img)
    # scharr_features = extract_scharr_features(img)

    features = {
        # "hog1" : hog_features[0], "hog2" : hog_features[1], "hog3" : hog_features[2], "hog4" : hog_features[3], 
        # "hog5" : hog_features[4], "hog6" : hog_features[5], "hog7" : hog_features[6], "hog8" : hog_features[7], 

        # "lbp1" : lbp_features[0], "lbp2" : lbp_features[1], "lbp3" : lbp_features[2], "lbp4" : lbp_features[3],
        # "lbp5" : lbp_features[4], "lbp6" : lbp_features[5], "lbp7" : lbp_features[6], "lbp8" : lbp_features[7],

        # "canny1" : canny_features[0], "canny2" : canny_features[1], "canny3" : canny_features[2], "canny4" : canny_features[3],
        # "canny5" : canny_features[4], "canny6" : canny_features[5], "canny7" : canny_features[6], "canny8" : canny_features[7],

        "bounding_box_x" : bounding_box[0], "bounding_box_y" : bounding_box[1], "bounding_box_w" : bounding_box[2], 
        "bounding_box_h" : bounding_box[3],

        "silhouette1" : silhouette[0], "silhouette2" : silhouette[1], "silhouette3" : silhouette[2], "silhouette4" : silhouette[3],
        "silhouette5" : silhouette[4], "silhouette6" : silhouette[5], "silhouette7" : silhouette[6], "silhouette8" : silhouette[7],
        "silhouette9" : silhouette[8], "silhouette10" : silhouette[9], "silhouette11" : silhouette[10], "silhouette12" : silhouette[11],
        "silhouette13" : silhouette[12], "silhouette14" : silhouette[13], "silhouette15" : silhouette[14], "silhouette16" : silhouette[15],
        "silhouette17" : silhouette[16], "silhouette18" : silhouette[17], "silhouette19" : silhouette[18], "silhouette20" : silhouette[19],

        "red1" : red[0], "red2" : red[1], "red3" : red[2], "red4" : red[3], "red5" : red[4], "red6" : red[5], "red7" : red[6],
        "red8" : red[7], "red9" : red[8], "red10" : red[9], "red11" : red[10], "red12" : red[11], "red13" : red[12], "red14" : red[13],
        "red15" : red[14], "red16" : red[15], "red17" : red[16], "red18" : red[17], "red19" : red[18], "red20" : red[19],
        "green1" : green[0], "green2" : green[1], "green3" : green[2], "green4" : green[3], "green5" : green[4], "green6" : green[5],
        "green7" : green[6], "green8" : green[7], "green9" : green[8], "green10" : green[9], "green11" : green[10], "green12" : green[11],
        "green13" : green[12], "green14" : green[13], "green15" : green[14], "green16" : green[15], "green17" : green[16], "green18" : green[17],
        "green19" : green[18], "green20" : green[19],
        "blue1" : blue[0], "blue2" : blue[1], "blue3" : blue[2], "blue4" : blue[3], "blue5" : blue[4], "blue6" : blue[5], "blue7" : blue[6],
        "blue8" : blue[7], "blue9" : blue[8], "blue10" : blue[9], "blue11" : blue[10], "blue12" : blue[11], "blue13" : blue[12], "blue14" : blue[13],
        "blue15" : blue[14], "blue16" : blue[15], "blue17" : blue[16], "blue18" : blue[17], "blue19" : blue[18], "blue20" : blue[19]

        # "sift1" : sift_features[0], "sift2" : sift_features[1], "sift3" : sift_features[2], "sift4" : sift_features[3],
        # "sift5" : sift_features[4], "sift6" : sift_features[5], "sift7" : sift_features[6], "sift8" : sift_features[7],

        # "hu1" : hu_moments[0], "hu2" : hu_moments[1], "hu3" : hu_moments[2], "hu4" : hu_moments[3], "hu5" : hu_moments[4],
        # "hu6" : hu_moments[5], "hu7" : hu_moments[6], "hu8" : hu_moments[7],

        # "fourier1" : fourier_transform[0], "fourier2" : fourier_transform[1], "fourier3" : fourier_transform[2], 
        # "fourier4" : fourier_transform[3], "fourier5" : fourier_transform[4], "fourier6" : fourier_transform[5],
        # "fourier7" : fourier_transform[6], "fourier8" : fourier_transform[7], 

        # "scharr1" : scharr_features[0], "scharr2" : scharr_features[1], "scharr3" : scharr_features[2], "scharr4" : scharr_features[3],
        # "scharr5" : scharr_features[4], "scharr6" : scharr_features[5], "scharr7" : scharr_features[6], "scharr8" : scharr_features[7],
    }
    print("done:", file)

    new_data.append(features)

done: 000000.jpg
done: 000001.jpg
done: 000002.jpg
done: 000003.jpg
done: 000004.jpg
done: 000005.jpg
done: 000006.jpg
done: 000007.jpg
done: 000008.jpg


KeyboardInterrupt: 

In [ ]:
new_data = pd.DataFrame(new_data)
new_data['id'] = data['id']
new_data['Category'] = data['Category']

new_data

In [ ]:
cols = new_data.columns.tolist()
cols = cols[-2:] + cols[:-2]
new_data = new_data[cols]

In [ ]:
new_data.to_csv('data/features.csv', index=False) 

In [20]:
for file in os.listdir('resized_images'):
    os.remove('resized_images/' + file)
os.rmdir('resized_images')